# Notebook 4: Projecting Injections

In this notebook we will explore how we can use GravyFlow GPU functions to project injections onto the GPU with physically in a physically realistic fashion.

In [1]:
# Built-in imports
from typing import List
from pathlib import Path

# Dependency imports: 
import numpy as np
import tensorflow as tf
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot

# Import the GravyFlow module.
import gravyflow as gf

2024-02-28 08:22:59.710333: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 08:22:59.710400: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 08:22:59.711598: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-28 08:22:59.719007: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-28 08:23:01.038620: W tensorflow/compiler/tf2

In [2]:
# Set up the environment using gf.env() and return a tf.distribute.Strategy object.
env : tf.distribute.Strategy = gf.env()

INFO:root:TensorFlow version: 2.15.0, CUDA version: 12.2
2024-02-28 08:23:16.370558: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-02-28 08:23:16.370983: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2000 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:07:00.0, compute capability: 7.0
INFO:root:[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
phenom_d_generator : gf.WaveformGenerator = gf.cuPhenomDGenerator(
    mass_1_msun=50.0,
    mass_2_msun=50.0,
    inclination_radians=0.0
)

phenom_d_injection_generator : gf.InjectionGenerator = gf.InjectionGenerator(phenom_d_generator)

# Use the TensorFlow environment 'env' created earlier with gf.env()
with env:
    phenom_d_injection, _, _ = next(phenom_d_injection_generator(num_examples_per_batch=1))

2024-02-28 08:23:17.278967: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-28 08:23:18.313162: I external/local_xla/xla/service/service.cc:168] XLA service 0x5570a76af820 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-28 08:23:18.313189: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-16GB, Compute Capability 7.0
2024-02-28 08:23:18.350881: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1709137398.368725  241088 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


## Projecting Injections onto Multiple Detectors

In [4]:
network : gf.Network = gf.Network([gf.IFO.L1, gf.IFO.H1, gf.IFO.V1])

2024-02-28 08:23:18.865580: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


In [6]:
with env:
    projected_injections : tf.Tensor = network.project_wave(
        phenom_d_injection[0],
        right_ascension=0.0,
        declination=0.0,
        polarization=0.0
    )

2024-02-28 08:23:32.155294: I external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:326] ptxas warning : Registers are spilled to local memory in function 'fusion', 4 bytes spill stores, 4 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'fusion_1', 16 bytes spill stores, 12 bytes spill loads



In [7]:
projected_injection : tf.Tensor = projected_injections[0]

projection_layout : List = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

Loading BokehJS ...

In [8]:
# Random direction and polarisation
with env:
    projected_injections : tf.Tensor = network.project_wave(
        phenom_d_injection[0]
    )

projected_injection : tf.Tensor = projected_injections[0]
projection_layout : List = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

2024-02-28 08:23:40.511886: I external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:326] ptxas warning : Registers are spilled to local memory in function 'fusion', 4 bytes spill stores, 4 bytes spill loads



Loading BokehJS ...

In [9]:
# Adding Custom Detectors:

# Create from dictionary:
network = gf.Network({
    "longitude_radians" : [np.pi/2, np.pi/4], 
    "latitude_radians" : [-np.pi/4, np.pi/6],
    "y_angle_radians" : [(2*np.pi/3), np.pi], 
    "x_angle_radians" : None, 
    "height_meters" : [0.0, 0.0],
    "x_length_meters" : [4000.0, 10000.0],
    "y_length_meters" : [4000.0, 10000.0]
})

# Random direction and polarisation
with env:
    projected_injections : tf.Tensor = network.project_wave(
        phenom_d_injection[0]
    )

projected_injection : tf.Tensor = projected_injections[0]

projection_layout = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

2024-02-28 08:23:48.616968: I external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:326] ptxas warning : Registers are spilled to local memory in function 'fusion', 4 bytes spill stores, 4 bytes spill loads



Loading BokehJS ...

## Load Detector From Config

In [10]:
# Define injection directory path:
example_network_directory : Path =  Path("./example_configs/example_network.json")

network = gf.Network.load(example_network_directory)

# Random direction and polarisation
with env:
    projected_injections : tf.Tensor = network.project_wave(
        phenom_d_injection[0]
    )

projected_injection : tf.Tensor = projected_injections[0]

projection_layout = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

Loading BokehJS ...

## Incoherent Injections

In [12]:
wnb_generator_a : gf.WaveformGenerator = gf.WNBGenerator(
    duration_seconds=0.7,
    min_frequency_hertz=50.0,
    max_frequency_hertz=100.0
)
phenom_d_generator_a : gf.WaveformGenerator = gf.cuPhenomDGenerator(
    mass_1_msun=50.0,
    mass_2_msun=50.0,
    inclination_radians=10.0
)
phenom_d_generator_b : gf.WaveformGenerator = gf.cuPhenomDGenerator(
    mass_1_msun=10.0,
    mass_2_msun=10.0,
    inclination_radians=20.0
)

incoherent_generator : gf.InjectionGenerator  = gf.IncoherentGenerator(
    [wnb_generator_a, phenom_d_generator_a, phenom_d_generator_b]
)

incoherent_injection_generator : gf.InjectionGenerator = gf.InjectionGenerator(incoherent_generator)    

In [13]:
network = gf.Network([gf.IFO.L1, gf.IFO.H1, gf.IFO.H1])
# Use the TensorFlow environment 'env' created earlier with gf.env()
with env:
    incoherent_injections, _, _ = next(
        incoherent_injection_generator(num_examples_per_batch=1)
    )        

    projected_injections : tf.Tensor = network.project_wave(
        incoherent_injections
    )

projected_injection : tf.Tensor = projected_injections[0]

projection_layout = [
    [gf.generate_strain_plot(
        {"Injection Test": injection},
        title=f"Injection projection example"    
    )]
    for injection in projected_injection
]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(projection_layout)
output_notebook()
show(grid)

tf.Tensor(
[[[[[-0.         -0.         -0.         ...  0.
      0.          0.        ]
    [ 0.          0.          0.         ...  0.
      0.          0.        ]]

   [[-0.51986265 -0.49356255 -0.46467903 ...  0.
      0.          0.        ]
    [-0.3477014  -0.38433105 -0.41898462 ...  0.
      0.          0.        ]]

   [[-0.0853213  -0.08162809 -0.07459102 ...  0.
      0.          0.        ]
    [-0.00974463 -0.02646062 -0.04272318 ...  0.
      0.          0.        ]]]]], shape=(1, 1, 3, 2, 4096), dtype=float32)
tf.Tensor(
[[[[4.7730282e-09 1.2805685e-09 1.7462298e-09 ...
    7.4146298e-09 8.1677509e-09 6.0899987e-09]
   [1.1441666e-01 1.3117941e-01 1.4619844e-01 ...
    6.3397765e-02 8.0250680e-02 9.8262340e-02]
   [3.0348230e-02 3.3511374e-02 3.4733307e-02 ...
    1.5333397e-02 2.0958103e-02 2.6561528e-02]]]], shape=(1, 1, 3, 4096), dtype=float32)


2024-02-28 08:24:08.598966: I external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:326] ptxas warning : Registers are spilled to local memory in function 'fusion', 4 bytes spill stores, 4 bytes spill loads



Loading BokehJS ...